In [7]:
%load_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pandas as pd
from cuml.manifold.umap import UMAP as cumlUMAP
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir
from joblib import Parallel, delayed

In [9]:
import pomegranate
from pomegranate import DiscreteDistribution, HiddenMarkovModel
pomegranate.utils.disable_gpu()
from hmmlearn import hmm

### Fit models code

In [17]:
def AIC(log_likelihood, k):
    """ AIC given log_likelihood and # parameters (k)
    """
    aic = 2 * k - 2 * log_likelihood
    return aic


def BIC(log_likelihood, n, k):
    """ BIC given log_likelihood, number of observations (n) and # parameters (k)
    """
    bic = np.log(n) * k - 2 * log_likelihood
    return bic

def fit_hmm(
    seqs,
    n_components,
    prop_test=0.5,
    verbose=False,
    pom_stop_threshold=1e-8,
    pom_max_iterations=1e8,
):

    # split train and test
    if prop_test == 0:
        seqs_train = seqs_test = seqs
    else:
        # split into train and test for cross validation
        training_mask = np.random.choice(
            np.arange(len(seqs)), size=int(len(seqs) * prop_test), replace=False
        )
        testing_mask = np.array(
            [i for i in np.arange(len(seqs)) if i not in training_mask]
        )

        seqs_train = np.array(seqs)[training_mask]
        seqs_test = np.array(seqs)[testing_mask]

        # make sure test set doesn't contain any data that train doesnt
        assert np.all(
            [
                i in np.unique(np.concatenate(seqs_train))
                for i in np.unique(np.concatenate(seqs_test))
            ]
        )

    # lengths of sequences
    seq_lens = [len(i) for i in seqs_train]

    # get states
    unique_states = np.unique(np.concatenate(seqs_train))

    # model
    hmmlearn_model = hmm.MultinomialHMM(n_components=n_components, verbose=verbose).fit(
        np.concatenate(seqs_train).reshape(-1, 1), [len(i) for i in seqs_train]
    )

    # probability of data given model
    hmmlearn_log_probability = hmmlearn_model.score(
        np.concatenate(seqs_test).reshape(-1, 1), [len(i) for i in seqs_test]
    )

    # hidden labels
    hmmlearn_predictions = [
        hmmlearn_model.predict(np.array(seq).reshape(-1, 1)) for seq in seqs
    ]

    # probability of ending in state, for each state (for pom)
    hmmlearn_end_states = np.array([i[-1] for i in hmmlearn_predictions])
    end_probs = [
        np.sum(hmmlearn_end_states == i)
        / (np.sum(np.concatenate(hmmlearn_predictions) == i) + 1)
        for i in np.arange(n_components)
    ]

    # initialize pom
    transmat = hmmlearn_model.transmat_
    start_probs = hmmlearn_model.startprob_
    dists = hmmlearn_model.emissionprob_

    states = [
        DiscreteDistribution({vis: d[i] for i, vis in enumerate(unique_states)})
        for d in dists
    ]
    pom_model = HiddenMarkovModel.from_matrix(
        transition_probabilities=transmat,
        distributions=states,
        starts=start_probs,
        ends=end_probs,  # discluding ends and merge makes models equal log prob
        merge="None",
    )

    # fit pom model
    improvement = pom_model.fit(
        seqs_train,
        max_iterations=pom_max_iterations,
        verbose=verbose,
        algorithm="baum-welch",
        stop_threshold=pom_stop_threshold,
        n_jobs=1,
    )

    pom_log_probability = np.sum([pom_model.log_probability(seq) for seq in seqs_test])

    pom_predictions = [pom_model.predict(seq) for seq in seqs]

    num_params = pom_model.state_count() + pom_model.edge_count() + pom_model.node_count()
    
    n_data = np.sum([len(i) for i in seqs_test])
    
    aic = AIC(pom_log_probability, num_params)
    bic = BIC(pom_log_probability, n_data, num_params)

    return (
        n_components,
        hmmlearn_model,
        pom_model,
        seqs_train,
        seqs_test,
        hmmlearn_predictions,
        pom_predictions,
        num_params,
        hmmlearn_log_probability,
        pom_log_probability,
        aic,
        bic,
    )

In [18]:
DATASET_ID = 'koumura_bengalese_finch'
embeddings_dfs = list(DATA_DIR.glob('bf_label_dfs/'+DATASET_ID+'/*.pickle'))
DATASET_ID = 'bengalese_finch_sober'
embeddings_dfs = embeddings_dfs + list(DATA_DIR.glob('bf_label_dfs/'+DATASET_ID+'/*.pickle'))

In [19]:
embeddings_dfs[:3]

[PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/bf_label_dfs/koumura_bengalese_finch/Bird9.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/bf_label_dfs/koumura_bengalese_finch/Bird4.pickle'),
 PosixPath('/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/bf_label_dfs/koumura_bengalese_finch/Bird10.pickle')]

In [21]:
for loc in tqdm(embeddings_dfs):
    print(indv)
    # read dataframe
    indv_df = pd.read_pickle(loc).sort_values(by=["key", "start_time"])
    indv = indv_df.indv.unique()[0]

    # Get seqs
    hand_seqs = [
        list(indv_df[indv_df.syllables_sequence_id == seqid]["labels_num"].values)
        for seqid in indv_df.syllables_sequence_id.unique()
    ]

    
    n_repeats = 5
    results = Parallel(n_jobs=-1, verbose=15)(
        delayed(fit_hmm)(
            hand_seqs,
            n_components,
            prop_test = 0.0,
            verbose=False,
            pom_stop_threshold=1e-1,
            pom_max_iterations=1e3,
        )
        for n_components in tqdm(
            np.random.permutation(np.repeat(np.arange(15, 45, 2), n_repeats))
        )
    )
    results_df = pd.DataFrame(
        results,
        columns=[
            "n_components",
            "hmmlearn_model",
            "pom_model",
            "seqs_train",
            "seqs_test",
            "hmmlearn_predictions",
            "pom_predictions",
            "n_params",
            "hmmlearn_log_probability",
            "pom_log_probability",
            "aic",
            "bic",
        ],
    )
    
    results_df["indv"] = indv
    save_loc = DATA_DIR / "HMM_fits" / "HMM" / (indv + ".pickle")
    ensure_dir(save_loc)
    results_df.to_pickle(save_loc)

    

Bird9


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  18 out of  75 | elapsed:  4.6min remaining: 14.6min
[Parallel(n_jobs=-1)]: Done  24 out of  75 | elapsed:  5.5min remaining: 11.6min
[Parallel(n_jobs=

Bird9


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   33.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   40.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   49.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   55.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  18 out of  75 | elapsed:  1.4min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  24 out of  75 | elapsed:  1.8min remaining:  3.8min
[Parallel(n_jobs=

Bird4


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  18 out of  75 | elapsed:  5.7min remaining: 18.2min
[Parallel(n_jobs=-1)]: Done  24 out of  75 | elapsed:  6.7min remaining: 14.2min
[Parallel(n_jobs=

gy6or6


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  18 out of  75 | elapsed:  6.6min remaining: 21.0min
[Parallel(n_jobs=-1)]: Done  24 out of  75 | elapsed:  8.1min remaining: 17.2min
[Parallel(n_jobs=

or60yw70


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done  18 out of  75 | elapsed: 12.6min remaining: 39.8min
[Parallel(n_jobs=-1)]: Done  24 out of  75 | elapsed: 14.4min remaining: 30.7min
[Parallel(n_jobs=